# 📚 EXIST2025: Sistema Híbrido para Detección y Análisis de Sexismo en Tweets

Este notebook aborda el reto de la competición EXIST2025, centrada en la detección automática y el análisis del sexismo en mensajes de Twitter. El objetivo es construir un sistema robusto que combine modelos de lenguaje grandes (LLMs) y modelos de transformers locales para abordar tres tareas principales:

---

## 🔎 **Tarea 1.1: Detección Binaria de Sexismo**
**Reto:** Determinar si un tweet es sexista o no.
- **Predicción:** `YES` (sexista) o `NO` (no sexista).
- **Intención:** Proveer una clasificación precisa y confiable, combinando el conocimiento de modelos LLM y la especialización de transformers.

---

## 🧠 **Tarea 1.2: Clasificación de la Intención**
**Reto:** Identificar la intención detrás del mensaje sexista.
- **Clases:** `DIRECT`, `REPORTED`, `JUDGEMENTAL`, `NO`.
- **Intención:** Analizar el contexto y matiz del mensaje para entender si el sexismo es directo, reportado, de juicio, o ausente.

---

## 🏷️ **Tarea 1.3: Categorización del Tipo de Sexismo**
**Reto:** Asignar una o más categorías específicas al sexismo detectado.
- **Categorías:** `IDEOLOGICAL-INEQUALITY`, `STEREOTYPING-DOMINANCE`, `OBJECTIFICATION`, `SEXUAL-VIOLENCE`, `MISOGYNY-NON-SEXUAL-VIOLENCE`.
- **Intención:** Ofrecer un análisis detallado sobre la naturaleza y manifestación del sexismo en el tweet.

---

## ⚙️ **Intención General del Notebook**

Este notebook implementa un sistema híbrido que:
- Procesa y estructura los datos de entrenamiento.
- Indexa ejemplos y criterios en un motor vectorial (Qdrant) para recuperación eficiente.
- Utiliza RAG (Retrieval-Augmented Generation) para enriquecer los prompts de los LLMs con ejemplos y definiciones relevantes.
- Combina predicciones de transformers locales y LLMs remotos mediante lógica de ensemble para mejorar la robustez y explicabilidad.
- Evalúa el rendimiento del sistema en las tres tareas, proporcionando métricas y visualizaciones.

El enfoque busca maximizar la precisión y la interpretabilidad, integrando lo mejor de ambos mundos: la potencia de lIos LLMs y la especialización de los modelos entrenados localmente.

In [9]:
# PASO 1: Procesamiento de datos para los 3 RAGs
import pandas as pd
import json
from collections import Counter
from llama_index.core import Document

def process_data_for_all_rags(file_path):
    """Procesa el archivo CSV y devuelve listas de Documentos para las tres tareas para RAGs.
    """
    docs_1_1, docs_1_2, docs_1_3 = [], [], []

    df = pd.read_csv(file_path)
    print(df.head())
    print(df.info())
    print(df.columns.tolist())

    for idx, row in df.iterrows():
        if 'TRAIN' not in row['split']:
            continue
        text = row['tweet']
        lang = row['lang']
        id_exist = row['id_EXIST']

        # --- Tarea 1.1: Sexism ---
        labels_1_1 = row['labels_task1_1']
        if isinstance(labels_1_1, str):
            labels_1_1 = json.loads(labels_1_1.replace("'", '"'))
        majority_label_1_1 = Counter(labels_1_1).most_common(1)[0][0]
        metadata_1_1 = {"id_EXIST": id_exist, "lang": lang, "label": majority_label_1_1}
        docs_1_1.append(Document(text=text, metadata=metadata_1_1))

        if majority_label_1_1 == "NO":
            metadata_1_2 = {"id_EXIST": id_exist, "lang": lang, "label": "NO"}
            docs_1_2.append(Document(text=text, metadata=metadata_1_2))

            metadata_1_3 = {"id_EXIST": id_exist, "lang": lang, "labels": ["NO"]}
            docs_1_3.append(Document(text=text, metadata=metadata_1_3))
        else:
            # Tarea 1.2
            labels_1_2 = row['labels_task1_2']
            if isinstance(labels_1_2, str):
                labels_1_2 = json.loads(labels_1_2.replace("'", '"'))
            labels_1_2 = [l for l in labels_1_2 if l != "-"]
            if labels_1_2:
                majority_label_1_2 = Counter(labels_1_2).most_common(1)[0][0]
                metadata_1_2 = {"id_EXIST": id_exist, "lang": lang, "label": majority_label_1_2}
                docs_1_2.append(Document(text=text, metadata=metadata_1_2))

            # Tarea 1.3
            labels_1_3 = row['labels_task1_3']
            if isinstance(labels_1_3, str):
                labels_1_3 = json.loads(labels_1_3.replace("'", '"'))
            labels_1_3_raw = []
            for sublist in labels_1_3:
                if isinstance(sublist, list):
                    labels_1_3_raw.extend([cat for cat in sublist if cat != "-"])
                elif sublist != "-":
                    labels_1_3_raw.append(sublist)
            if labels_1_3_raw:
                vote_counts_1_3 = Counter(labels_1_3_raw)
                final_labels_1_3 = [label for label, count in vote_counts_1_3.items() if count > 1]
                if final_labels_1_3:
                    metadata_1_3 = {"id_EXIST": id_exist, "lang": lang, "labels": final_labels_1_3}
                    docs_1_3.append(Document(text=text, metadata=metadata_1_3))

    return docs_1_1, docs_1_2, docs_1_3

In [10]:
docs_1_1, docs_1_2, docs_1_3 = process_data_for_all_rags('../data/dataset_task1_2_limpio.csv')

   id_EXIST lang                                              tweet  \
0    100001   es  @TheChiflis Ignora al otro, es un capullo.El p...   
1    100002   es  @ultimonomada_ Si comicsgate se parece en algo...   
2    100003   es  @Steven2897 Lee sobre Gamergate, y como eso ha...   
3    100004   es  @Lunariita7 Un retraso social bastante lamenta...   
4    100005   es  @novadragon21 @icep4ck @TvDannyZ Entonces como...   

   number_annotators                                         annotators  \
0                  6  ['Annotator_1', 'Annotator_2', 'Annotator_3', ...   
1                  6  ['Annotator_7', 'Annotator_8', 'Annotator_9', ...   
2                  6  ['Annotator_7', 'Annotator_8', 'Annotator_9', ...   
3                  6  ['Annotator_13', 'Annotator_14', 'Annotator_15...   
4                  6  ['Annotator_19', 'Annotator_20', 'Annotator_21...   

                gender_annotators  \
0  ['F', 'F', 'F', 'M', 'M', 'M']   
1  ['F', 'F', 'F', 'M', 'M', 'M']   
2  ['F', 'F

In [11]:
import llama_index.embeddings
print(dir(llama_index.embeddings))

['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


## 🧠 Indexación y Recuperación Semántica con Qdrant, Embeddings y LLMs (Decoupled)

En las siguientes celdas se implementa la infraestructura técnica clave para el sistema híbrido EXIST2025, integrando motores de búsqueda semántica, embeddings avanzados y modelos de lenguaje grandes (LLMs) de manera desacoplada ("decoupled"). A continuación se resume el flujo y los componentes principales:

---

### 1. **Indexación Vectorial con Qdrant**

- **Qdrant** es un motor de base de datos vectorial que permite almacenar y buscar documentos en función de su similitud semántica.
- Se crean colecciones separadas en Qdrant para cada sub-tarea (`task1_1_rag`, `task1_2_rag`, `task1_3_rag` y `criterios_rag`), permitiendo búsquedas eficientes y especializadas.
- Los documentos (`Document`) se indexan junto con metadatos relevantes (por ejemplo, etiquetas, idioma, sub-tarea).

---

### 2. **Generación de Embeddings**

- Se utilizan dos tipos de modelos de embeddings:
    - **OllamaEmbedding**: Embeddings generados mediante un modelo alojado en un servidor Ollama remoto, ideal para alinearse con los LLMs utilizados en inferencia.
    - **FastEmbedEmbedding**: Embeddings rápidos y eficientes para tareas de recuperación local.
- Los embeddings permiten transformar textos en vectores numéricos que capturan su significado semántico, facilitando la búsqueda por similitud.

---

### 3. **Recuperación de Ejemplos y Criterios vía RAG**

- Se implementa un sistema RAG (Retrieval-Augmented Generation) que, antes de consultar al LLM, recupera ejemplos y criterios relevantes desde los índices vectoriales.
- Esto enriquece los prompts enviados al LLM, mejorando la precisión y explicabilidad de las respuestas.

---

### 4. **Inferencia con LLMs Desacoplados (Decoupled)**

- La función `prompt_decoupled` permite enviar prompts enriquecidos a un LLM remoto (por ejemplo, `mistral-nemo:latest`) alojado en un servidor Ollama externo.
- El sistema desacopla la recuperación semántica (Qdrant + embeddings) de la generación de texto (LLM), permitiendo escalabilidad y modularidad.
- Se procesan las respuestas del LLM para extraer predicciones estructuradas (por ejemplo, en formato JSON).

---

### **Resumen del Flujo Técnico**

1. **Indexación**: Los documentos y criterios se almacenan en Qdrant como vectores.
2. **Recuperación**: Ante una consulta, se buscan los ejemplos y criterios más relevantes usando embeddings.
3. **Generación**: Se construye un prompt enriquecido y se envía al LLM remoto usando una arquitectura desacoplada.
4. **Respuesta**: El LLM responde en formato estructurado, facilitando la integración con el sistema híbrido y la evaluación automática.

---

Este enfoque permite combinar lo mejor de la búsqueda semántica y la generación de lenguaje natural, optimizando tanto la precisión como la interpretabilidad del sistema para la detección y análisis de sexismo en tweets.

In [12]:
import sys
import logging
import qdrant_client

from IPython.display import Markdown, display
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama




# (Opcional) Configurar logging para ver más detalles del proceso
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#documents = SimpleDirectoryReader("./data_pdf/").load_data()


client = qdrant_client.QdrantClient(
    url='https://qdrant01.decoupled.ai:443',
    timeout=60,
    api_key=None,
)

ollama_embedding = OllamaEmbedding(
    model_name="mxbai-embed-large",
    base_url="https://ollama03.decoupled.ai",
    ollama_additional_kwargs={"mirostat": 0},
)

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.embed_model = ollama_embedding
Settings.llm = Ollama(model="qwen2.5:latest", base_url="https://ollama01.decoupled.ai", request_timeout=60)

client = qdrant_client.QdrantClient(
    url='https://qdrant01.decoupled.ai:443',
    timeout=60,
    api_key=None,
)

print("¡Indexación completada con éxito!")

# Indexar Tarea 1.1
collectionLoaded = client.get_collections().collections
collection_names = [col.name for col in collectionLoaded]
print(f"📚 Colecciones actuales en Qdrant: {collection_names}")



# Indexar Tarea 1.1
if "task1_1_rag" not in collection_names:
    vector_store_1_1 = QdrantVectorStore(client=client, collection_name="task1_1_rag")
    storage_context_1_1 = StorageContext.from_defaults(vector_store=vector_store_1_1)
    index_1_1 = VectorStoreIndex.from_documents(docs_1_1, storage_context=storage_context_1_1, show_progress=True)
else:
    vector_store_1_1 = QdrantVectorStore(client=client, collection_name="task1_1_rag")
    storage_context_1_1 = StorageContext.from_defaults(vector_store=vector_store_1_1)
    index_1_1 = VectorStoreIndex.from_vector_store(vector_store=vector_store_1_1)

# Indexar Tarea 1.2
if "task1_2_rag" not in collection_names:
    vector_store_1_2 = QdrantVectorStore(client=client, collection_name="task1_2_rag")
    storage_context_1_2 = StorageContext.from_defaults(vector_store=vector_store_1_2)
    index_1_2 = VectorStoreIndex.from_documents(docs_1_2, storage_context=storage_context_1_2, show_progress=True)
else:
    vector_store_1_2 = QdrantVectorStore(client=client, collection_name="task1_2_rag")
    storage_context_1_2 = StorageContext.from_defaults(vector_store=vector_store_1_2)
    index_1_2 = VectorStoreIndex.from_vector_store(vector_store=vector_store_1_2)

# Indexar Tarea 1.3
if "task1_3_rag" not in collection_names:
    vector_store_1_3 = QdrantVectorStore(client=client, collection_name="task1_3_rag")
    storage_context_1_3 = StorageContext.from_defaults(vector_store=vector_store_1_3)
    index_1_3 = VectorStoreIndex.from_documents(docs_, storage_context=storage_context_1_3, show_progress=True)
else:
    vector_store_1_3 = QdrantVectorStore(client=client, collection_name="task1_3_rag")
    storage_context_1_3 = StorageContext.from_defaults(vector_store=vector_store_1_3)
    index_1_3 = VectorStoreIndex.from_vector_store(vector_store=vector_store_1_3)


INFO:httpx:HTTP Request: GET https://qdrant01.decoupled.ai "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai "HTTP/1.1 200 OK"


/tmp/ipykernel_68553/2444469068.py:23: UserWarning: Qdrant client version 1.15.1 is incompatible with server version 1.10.0. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = qdrant_client.QdrantClient(


INFO:httpx:HTTP Request: GET https://qdrant01.decoupled.ai "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai "HTTP/1.1 200 OK"
¡Indexación completada con éxito!
INFO:httpx:HTTP Request: GET https://qdrant01.decoupled.ai/collections "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai/collections "HTTP/1.1 200 OK"
📚 Colecciones actuales en Qdrant: ['task1_1_rag', 'task1_3_rag', 'task1_2_rag', 'criterios_rag']


/tmp/ipykernel_68553/2444469068.py:39: UserWarning: Qdrant client version 1.15.1 is incompatible with server version 1.10.0. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = qdrant_client.QdrantClient(


INFO:httpx:HTTP Request: GET https://qdrant01.decoupled.ai/collections/task1_1_rag/exists "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai/collections/task1_1_rag/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://qdrant01.decoupled.ai/collections/task1_1_rag "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai/collections/task1_1_rag "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://qdrant01.decoupled.ai/collections/task1_2_rag/exists "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai/collections/task1_2_rag/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://qdrant01.decoupled.ai/collections/task1_2_rag "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai/collections/task1_2_rag "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://qdrant01.decoupled.ai/collections/task1_3_rag/exists "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai/collections/task1_3_rag/exists "HTTP/1.1 200 OK"
INF

In [13]:
import json
from llama_index.core import Document

criterios_path = "../data/dataCriterios.json"
docs = []

# DOCUMENTOS PARA RAG CRITERIOS
if "criterios_rag" not in collection_names:

    with open(criterios_path, encoding="utf-8") as f:
        for line in f:
            frag = json.loads(line)
            # Puedes filtrar por subtask, tipo, idioma, etc. si lo deseas
            docs.append(Document(text=frag["content"], metadata={
                "subtask": frag["subtask"],
                "type": frag["type"],
                "lang": frag["lang"]
            }))

    print(f"Se han creado {len(docs)} documentos para el índice RAG.")
    print("Ejemplo de documento:", docs[0])

In [14]:
# 2. Crear el vector store y el contexto de almacenamiento
collection_name = "criterios_rag"
vector_store = QdrantVectorStore(client=client, collection_name=collection_name)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 3. Indexar los documentos (solo la primera vez, luego puedes cargar el índice)

if collection_name not in collection_names:
    rag_index = VectorStoreIndex.from_documents(
    docs,
    storage_context=storage_context,
    show_progress=True
    )
else:
  rag_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)







INFO:httpx:HTTP Request: GET https://qdrant01.decoupled.ai/collections/criterios_rag/exists "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai/collections/criterios_rag/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://qdrant01.decoupled.ai/collections/criterios_rag "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai/collections/criterios_rag "HTTP/1.1 200 OK"


## 📝 Ingeniería de Prompts y Recuperación Semántica (RAG) para EXIST2025

En esta sección del notebook se implementa y documenta el proceso avanzado de generación de prompts (prompting) para la interacción con modelos de lenguaje grandes (LLMs) en el contexto de la competición EXIST2025. El objetivo es maximizar la precisión y explicabilidad de las predicciones mediante técnicas de Recuperación Aumentada por Recuperación (RAG) y una ingeniería de prompts cuidadosamente diseñada.

  
**Nota técnica:**  
El sistema implementa tanto recuperación "zero-shot" (sin ejemplos explícitos) como "few-shot" (con ejemplos recuperados dinámicamente) según la configuración y la subtarea. Para tareas donde la generalización es prioritaria o los ejemplos son escasos, se puede optar por prompts zero-shot, apoyándose únicamente en definiciones y criterios. En cambio, para maximizar la precisión y el razonamiento contextual, se emplea recuperación few-shot, donde los ejemplos más similares al tweet de entrada son seleccionados automáticamente desde el índice vectorial y presentados en el prompt. Esta flexibilidad permite adaptar la estrategia de prompting a las necesidades específicas de cada subtarea y experimentar con variantes de ingeniería de prompts para optimizar el rendimiento del sistema.


### 1. **Recuperación de Criterios y Ejemplos Relevantes (RAG)**

Se utiliza un índice vectorial (`rag_index`) construido con Qdrant y embeddings semánticos para recuperar, de manera dinámica, criterios y definiciones relevantes para cada subtarea (1.1, 1.2, 1.3). Esto permite que cada prompt enviado al LLM esté enriquecido con información contextual específica, aumentando la robustez y la interpretabilidad de las respuestas.

- **Criterios y definiciones**: Se recuperan fragmentos de criterios y definiciones almacenados en el índice vectorial, filtrados por subtarea e idioma.
- **Ejemplos similares**: Se emplean recuperadores específicos para cada subtarea (`retriever_1_1`, `retriever_1_2`, `retriever_1_3`) que extraen ejemplos de tweets anotados similares al texto de entrada, facilitando el razonamiento contextual del modelo.

---

### 2. **Construcción Modular de Prompts**

Se define la clase `EXIST2025PromptBuilder`, que encapsula la lógica de construcción de prompts para cada subtarea:

- **Header contextual**: Incluye definiciones y criterios relevantes recuperados vía RAG.
- **Instrucción de auto-generación**: Se solicita al LLM que piense en ejemplos propios antes de responder, fomentando un razonamiento más profundo y comparativo.
- **Formato de respuesta estructurado**: Se exige al modelo responder en formato JSON, facilitando la extracción automática de predicciones y su integración en el pipeline de evaluación.

---

### 3. **Personalización por Subtarea**

La generación de prompts se adapta a cada subtarea:

- **Tarea 1.1 (Binaria)**: El prompt pregunta explícitamente si el tweet es sexista, aportando ejemplos y criterios relevantes.
- **Tarea 1.2 (Intención)**: El prompt guía al modelo a identificar la intención detrás del mensaje, diferenciando entre categorías como DIRECT, REPORTED, JUDGEMENTAL o NO.
- **Tarea 1.3 (Categorías de Sexismo)**: El prompt solicita al modelo identificar y listar las categorías específicas de sexismo presentes en el tweet, apoyándose en criterios y ejemplos pertinentes.

---

### 4. **Automatización y Flexibilidad**

El sistema permite generar prompts de manera automática y flexible para cualquier tweet y subtarea, asegurando consistencia y adaptabilidad en todo el flujo de inferencia. Además, la integración con los índices vectoriales garantiza que la información recuperada esté siempre alineada con el contexto y las necesidades de cada predicción.

---

### 5. **Ventajas de la Estrategia Adoptada**

- **Explicabilidad**: Los prompts enriquecidos con criterios y ejemplos permiten comprender mejor la base de cada predicción.
- **Robustez**: La recuperación semántica asegura que el modelo disponga de información relevante, incluso ante casos ambiguos o complejos.
- **Escalabilidad**: El enfoque modular facilita la extensión a nuevas tareas, idiomas o dominios.

---

En resumen, la ingeniería de prompts implementada en este notebook representa una solución profesional y avanzada para la interacción con LLMs en tareas de clasificación complejas, combinando lo mejor de la recuperación semántica y la generación controlada de lenguaje natural.

In [15]:
import json
from llama_index.core.retrievers import BaseRetriever

class EXIST2025PromptBuilder:
    """
    Prompt builder SOLO para uso con RAG (necesita rag_index).
    Recupera criterios y ejemplos relevantes dinámicamente para cadfa task.
    """
    def __init__(self, rag_index, retriever_1_1: BaseRetriever, retriever_1_2: BaseRetriever, retriever_1_3: BaseRetriever, language="es"):
        self.rag_index = rag_index
        self.language = language.lower()
        self.definitions = self._load_definitions()
        self.categories = [
            "IDEOLOGICAL-INEQUALITY",
            "STEREOTYPING-DOMINANCE",
            "OBJECTIFICATION",
            "SEXUAL-VIOLENCE",
            "MISOGYNY-NON-SEXUAL-VIOLENCE",
            "NO"
        ]
        self.retrievers_examples = {
            '1.1': retriever_1_1,
            '1.2': retriever_1_2,
            '1.3': retriever_1_3
        }

    def _load_definitions(self):
        """Carga las preguntas base según el idioma"""
        if self.language == "es":
            return {
                "key_question": "¿Es el tweet sexista de alguna forma, o describe situaciones donde ocurre discriminación?",
                "intention_question": "¿Cuál es la intención de la persona que escribió el tweet?",
                "categories_question": "Identifica QUÉ TIPO(S) de sexismo contiene este tweet:"
            }
        else:
            return {
                "key_question": "Is the tweet sexist in any form, or does it describe situations where discrimination occurs?",
                "intention_question": "What do you think the intention of the person that wrote the tweet is?",
                "categories_question": "Identify WHAT TYPE(S) of sexism this tweet contains:"
            }
        
    def _rag_header(self, subtask, tweet):
        retriever = self.rag_index.as_retriever(similarity_top_k=15)
        nodes = retriever.retrieve(tweet)
        # Filtra por subtask y lang
        nodes = [n for n in nodes if n.metadata.get("subtask") == subtask and n.metadata.get("lang") == self.language]
        definiciones = [n.get_content() for n in nodes if n.metadata.get("type") == "definition"]
        criterios = [n.get_content() for n in nodes if n.metadata.get("type") == "criteria"]
        # Selecciona la pregunta adecuada según el subtask
        if subtask == "1.1":
            pregunta = self.definitions["key_question"]
        elif subtask == "1.2":
            pregunta = self.definitions["intention_question"]
        elif subtask == "1.3":
            pregunta = self.definitions["categories_question"]
        else:
            pregunta = ""
        # Construye el header
        header = ""
        if definiciones:
            header += f"Definición: {definiciones[0]}\n"
        if criterios:
            header += "Criterios:\n" + "\n".join(f"- {c}" for c in criterios[:2]) + "\n"
        if pregunta:
            header += f"\nPREGUNTA: {pregunta}\n"
        header += f'\nTWEET: "{tweet}"\n'
        return header

    def _self_gen_instruction(self, subtask, label=None):
        """
        Instrucción para que el modelo piense en 3 ejemplos propios antes de responder.
        """
        if self.language == "es":
            if subtask == "1.1":
                return "Antes de responder, piensa en 3 ejemplos de tweets con contexto similar al de la pregunta (sexistas y no sexistas) y compáralos mentalmente."
            elif subtask == "1.2":
                return f"Antes de responder, piensa en 3 ejemplos de tweets con contexto similar al de la pregunta, con intención '{label}'  <NO, si no es sexista> y compáralos mentalmente."
            elif subtask == "1.3":
                return "Antes de responder, piensa en 3 ejemplos de tweets con contexto similar al de la pregunta para cada categoría  <NO, si no es sexista>  y compáralos mentalmente."
        else:
            if subtask == "1.1":
                return "Before answering, think of 3 tweets with a context similar to that of the question  (sexist and non-sexist) and compare them mentally."
            elif subtask == "1.2":
                return f"Before answering, think of 3 tweets with a context similar to that of the question with intention '{label}'  <NO, if non sexist> and compare them mentally."
            elif subtask == "1.3":
                return "Before answering, think of 3 tweets with a context similar to that of the question for each relevant category <NO, if non sexist> and compare them mentally."
        return ""
    
    def _build_examples(self, task: str, text: str) -> str:
        retrieved_nodes = self.retrievers_examples[task].retrieve(text)
        examples_string = ""
        for node in retrieved_nodes:
            examples_string += "---\n"
            examples_string += f"Example Post: {node.get_content()}\n"
            if task in ['1.1', '1.2']:
                examples_string += f"Correct Label: {node.metadata['label']}\n"
            elif task == '1.3':
                # El metadata para la Tarea 1.3 es una lista de etiquetas
                examples_string += f"Correct Labels: {json.dumps(node.metadata['labels'])}\n"
        return examples_string


    def build_binary_prompt(self, tweet):
        header = self._rag_header("1.1", tweet)
        instruccion = self._self_gen_instruction("1.1")
        return f"""{header}
            {instruccion}

            RESPUESTA: Formato JSON con "prediction" (YES/NO) y "confidence" (0.0 a 1.0)
            Responde ÚNICAMENTE con el objeto JSON.
            Ejemplo formato respuesta: {{"prediction": "prediction_value", "confidence": "confidence_value"}}
            """

    def build_intention_prompt(self, tweet):
        header = self._rag_header("1.2", tweet)
        instruccion = self._self_gen_instruction("1.2", label=
                                                 "DIRECT/REPORTED/JUDGEMENTAL/NO")
        return f"""{header}
        {instruccion}
        RESPUESTA:  Formato JSON con "prediction" (DIRECT/REPORTED/JUDGEMENTAL/NO <Categorizarlo como NO si no es sexista>) y "confidence" (0.0 a 1.0)"
        Responde ÚNICAMENTE con el objeto JSON.
        Ejemplo formato respuesta: {{"prediction": "prediction_value", "confidence": "confidence_value"}}


        """

    def build_categories_prompt(self, tweet):
        header = self._rag_header("1.3", tweet)
        instruccion = self._self_gen_instruction("1.3")
        return f"""{header}
        {instruccion}
        RESPUESTA:  Formato JSON con "prediction" ({self.categories} <Categorizarlo como NO si no es sexista>) y "confidence" (0.0 a 1.0)"
        Responde ÚNICAMENTE con el objeto JSON 
        Ejemplo formato respuesta: {{"prediction": "prediction_value", "confidence": "confidence_value"}}
        """

    def build_prompt(self, tweet, subtask="1.1"):
        examples = self._build_examples(subtask, tweet)

        if subtask == "1.1":
            return self.build_binary_prompt(tweet)
        elif subtask == "1.2":
            return self.build_intention_prompt(tweet)
        elif subtask == "1.3":
            return self.build_categories_prompt(tweet)
        else:
            raise ValueError(f"Subtask '{subtask}' no válida. Use '1.1', '1.2', o '1.3'")

# --- USO SOLO CON RAG ---
# Supón que tienes un objeto rag_index ya inicializado y funcional

# builder = EXIST2025PromptBuilder(rag_index, language="es")
# tweet = "Las mujeres no pueden manejar, llámame sexista pero es verdad"
# prompt = builder.build_prompt(tweet, subtask="1.1")
# print(prompt)

In [16]:
# 1. Carga tus índices de ejemplos (uno por subtask)
retriever_1_1 = index_1_1.as_retriever(similarity_top_k=3)
retriever_1_2 = index_1_2.as_retriever(similarity_top_k=3)
retriever_1_3 = index_1_3.as_retriever(similarity_top_k=3)

# 2. Carga tu índice de criterios (rag_index)
# rag_index ya debe estar creado como VectorStoreIndex

# 3. Crea el builder
prompt_builder = EXIST2025PromptBuilder(
    rag_index,
    language="es",
    retriever_1_1=retriever_1_1,
    retriever_1_2=retriever_1_2,
    retriever_1_3=retriever_1_3
)

# 4. Genera un prompt para cualquier subtask
tweet = "las mujeres son un amor"
prompt = prompt_builder.build_prompt(tweet, subtask="1.1")
print(prompt)



INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qdrant01.decoupled.ai/collections/task1_1_rag/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://qdrant01.decoupled.ai/collections/task1_1_rag/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qdrant01.decoupled.ai/collections/criterios_rag/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://qdrant01.decoupled.ai/collections/criterios_rag/points/search "HTTP/1.1 200 OK"
Criterios:
- La desigualdad de género se perpetúa cuando se normalizan frases como 'los hombres no lloran' o 'las mujeres son malas para la tecnología', aunque se digan en tono de broma.
- El sexismo puede estar presente aun

In [17]:
# Para subtask 1.2 o 1.3, solo cambia el argumento:
prompt2 = prompt_builder.build_prompt(tweet, subtask="1.2")
print(prompt2)



INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qdrant01.decoupled.ai/collections/task1_2_rag/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://qdrant01.decoupled.ai/collections/task1_2_rag/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qdrant01.decoupled.ai/collections/criterios_rag/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://qdrant01.decoupled.ai/collections/criterios_rag/points/search "HTTP/1.1 200 OK"
Criterios:
- La intención DIRECT implica que el mensaje es sexista por sí mismo, sin contexto adicional.
- La intención REPORTED describe o reporta una situación sexista, ya sea en primera o tercera persona.

PREGUNTA: ¿Cuál

In [18]:
prompt3 = prompt_builder.build_prompt(tweet, subtask="1.3")
print(prompt3)

INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qdrant01.decoupled.ai/collections/task1_3_rag/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://qdrant01.decoupled.ai/collections/task1_3_rag/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qdrant01.decoupled.ai/collections/criterios_rag/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://qdrant01.decoupled.ai/collections/criterios_rag/points/search "HTTP/1.1 200 OK"
Criterios:
- MISOGYNY-NON-SEXUAL-VIOLENCE: Mensajes que expresan odio, desprecio o violencia hacia las mujeres, sin connotación sexual.
- OBJECTIFICATION: Mensajes que reducen a las mujeres a su apariencia física o las prese

In [19]:
import json
import requests
from ollama import Client


def prompt_decoupled(query, model='mistral-nemo:latest', context=[]):

    host = 'https://ollama01.decoupled.ai'
    # host = 'http://localhost:11434'

    payload = {
        "prompt": query,
        "model": model,
        "stream": False,
        "context": context,
        "keep_alive": "1h"
    }

    try:
        messages = []
        message = {}
        message['role'] = "user" # system, assistant, user
        message['content'] = query
        messages.append(message)

        res = {}

        #res['response'], model = decoupled_rest(model, host, messages)
        res['response'], model = decoupled_python(model, host, messages)

        input_token_count = token_count(query)
        output_token_count = token_count(res['response'])

        if res['response'] is not None and res['response'] != '':
            return res, model, input_token_count, output_token_count
        else:
            print("EXCEPTION RES")
            return None, model, 0, 0

    except Exception as e:
        print("EXCEPTION: ", str(e))
        return None, model, 0, 0


def decoupled_rest(model, url, payload):
    headers = {
        "Content-Type": "application/json"
    }

    res = requests.post(url, headers=headers, data=json.dumps(payload), timeout=120)

    if res.status_code == 200:
        return res.text, model
    else:
        return None, model


def decoupled_python(model, url, messages):

    client = Client(host=url, timeout=120)
    response = client.chat(model=model, messages=messages, keep_alive="168h")

    return response['message']['content'], model


def token_count(phrase):
    tokens = 0

    for s in phrase:
        if '\n' in s or \
                '\t' in s or \
                ' ' in s:
            tokens += 1

    words = phrase.split()

    for word in words:
        count = syllable_count(word)
        tokens += count

    if tokens == 0:
        tokens = 1

    return tokens


def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiou"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if count == 0:
        count += 1
    return count


In [20]:
respuesta_1_3, modelo_usado_1_3, tokens_in_1_3, tokens_out_1_3 = prompt_decoupled(prompt, model='mistral-nemo:latest')

print("\nPrompt 1.1 enviado:")
print(prompt)
print(f"\nRespuesta 1.1:\n{respuesta_1_3['response']}")

INFO:httpx:HTTP Request: POST https://ollama01.decoupled.ai/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama01.decoupled.ai/api/chat "HTTP/1.1 200 OK"

Prompt 1.1 enviado:
Criterios:
- La desigualdad de género se perpetúa cuando se normalizan frases como 'los hombres no lloran' o 'las mujeres son malas para la tecnología', aunque se digan en tono de broma.
- El sexismo puede estar presente aunque el mensaje no mencione explícitamente a mujeres u hombres, por ejemplo, cuando se habla de 'personas que deberían quedarse en casa' en contextos laborales.

PREGUNTA: ¿Es el tweet sexista de alguna forma, o describe situaciones donde ocurre discriminación?

TWEET: "las mujeres son un amor"

            Antes de responder, piensa en 3 ejemplos de tweets con contexto similar al de la pregunta (sexistas y no sexistas) y compáralos mentalmente.

            RESPUESTA: Formato JSON con "prediction" (YES/NO) y "confidence" (0.0 a 1.0)
            Responde ÚNICAMENTE con el objeto JSON.
 

In [21]:
respuesta_1_2, modelo_usado_1_2, tokens_in_1_2, tokens_out_1_2 = prompt_decoupled(prompt2, model='mistral-nemo:latest')

print("\nPrompt 1.2 enviado:")
print(prompt2)
print(f"\nRespuesta 1.2:\n{respuesta_1_2['response']}")

INFO:httpx:HTTP Request: POST https://ollama01.decoupled.ai/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama01.decoupled.ai/api/chat "HTTP/1.1 200 OK"

Prompt 1.2 enviado:
Criterios:
- La intención DIRECT implica que el mensaje es sexista por sí mismo, sin contexto adicional.
- La intención REPORTED describe o reporta una situación sexista, ya sea en primera o tercera persona.

PREGUNTA: ¿Cuál es la intención de la persona que escribió el tweet?

TWEET: "las mujeres son un amor"

        Antes de responder, piensa en 3 ejemplos de tweets con contexto similar al de la pregunta, con intención 'DIRECT/REPORTED/JUDGEMENTAL/NO'  <NO, si no es sexista> y compáralos mentalmente.
        RESPUESTA:  Formato JSON con "prediction" (DIRECT/REPORTED/JUDGEMENTAL/NO <Categorizarlo como NO si no es sexista>) y "confidence" (0.0 a 1.0)"
        Responde ÚNICAMENTE con el objeto JSON.
        Ejemplo formato respuesta: {"prediction": "prediction_value", "confidence": "confidence_value"}


 

In [22]:
respuesta_1_3, modelo_usado_1_3, tokens_in_1_3, tokens_out_1_3 = prompt_decoupled(prompt3, model='mistral-nemo:latest')

print("\nPrompt 1.3 enviado:")
print(prompt3)
print(f"\nRespuesta 1.3:\n{respuesta_1_3['response']}")

INFO:httpx:HTTP Request: POST https://ollama01.decoupled.ai/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama01.decoupled.ai/api/chat "HTTP/1.1 200 OK"

Prompt 1.3 enviado:
Criterios:
- MISOGYNY-NON-SEXUAL-VIOLENCE: Mensajes que expresan odio, desprecio o violencia hacia las mujeres, sin connotación sexual.
- OBJECTIFICATION: Mensajes que reducen a las mujeres a su apariencia física o las presentan como objetos sexuales.

PREGUNTA: Identifica QUÉ TIPO(S) de sexismo contiene este tweet:

TWEET: "las mujeres son un amor"

        Antes de responder, piensa en 3 ejemplos de tweets con contexto similar al de la pregunta para cada categoría  <NO, si no es sexista>  y compáralos mentalmente.
        RESPUESTA:  Formato JSON con "prediction" (['IDEOLOGICAL-INEQUALITY', 'STEREOTYPING-DOMINANCE', 'OBJECTIFICATION', 'SEXUAL-VIOLENCE', 'MISOGYNY-NON-SEXUAL-VIOLENCE', 'NO'] <Categorizarlo como NO si no es sexista>) y "confidence" (0.0 a 1.0)"
        Responde ÚNICAMENTE con el objeto JS

## 🤖 Integración de Modelos Transformers Finos y Ensamble Híbrido con LLM

En esta sección se documenta la carga y utilización de modelos transformers previamente entrenados y optimizados para las tareas de EXIST2025. Estos modelos han sido afinados mediante técnicas avanzadas de limpieza de datos, congelación de capas y regularización (dropout), asegurando una mayor robustez y generalización en la detección y análisis de sexismo en tweets.

### 🔬 Carga de Modelos Finos

Se emplean arquitecturas basadas en `xlm-roberta-large`, adaptadas tanto para clasificación binaria (sexismo SÍ/NO) como para clasificación multiclase (intención del mensaje). Los modelos son cargados desde checkpoints entrenados, utilizando formatos eficientes como `safetensors` para una inicialización rápida y segura. Además, se asegura la compatibilidad con GPU para acelerar la inferencia.

### ⚡ Ensamble Híbrido: Transformer + LLM

El sistema implementa un enfoque de ensamble híbrido que combina la especialización de los modelos transformers locales con la capacidad contextual y generativa de los LLMs remotos. La lógica de ensamble pondera las predicciones de ambos modelos, priorizando la robustez y la explicabilidad:

- **Transformers locales:** Proveen decisiones rápidas y especializadas, aprovechando el aprendizaje supervisado sobre el dominio específico.
- **LLMs remotos:** Aportan razonamiento contextual y generalización, enriquecidos mediante prompts avanzados y recuperación semántica (RAG).

La integración de ambos enfoques permite maximizar la precisión, minimizar sesgos y ofrecer un sistema explicable y escalable para la detección y análisis de sexismo en redes sociales.

---

In [23]:
# 🔧 SISTEMA HÍBRIDO: TRANSFORMER + LLM ENSEMBLE

import sys
import torch.nn as nn
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, Trainer,AutoModel
from transformers.modeling_outputs import SequenceClassifierOutput
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import json
from typing import Dict, List, Tuple, Optional
import os

import torch

# Detectar si hay una GPU disponible (CUDA) y seleccionarla, de lo contrario, usar CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Dispositivo seleccionado: {device}")

if torch.cuda.is_available():
    print(f"🔥 Nombre de la GPU: {torch.cuda.get_device_name(0)}")

✅ Dispositivo seleccionado: cuda
🔥 Nombre de la GPU: NVIDIA GeForce RTX 4080 Laptop GPU


In [24]:
# CELDA [38] CORREGIDA

class SoftBinaryClassifier(nn.Module):
    MODEL_NAME = "xlm-roberta-large" #microsoft/mdeberta-v3-base"
    def __init__(self, model_name: str =MODEL_NAME, dropout: float = 0.3):
        """
        Inicializa la arquitectura del modelo a partir de un nombre de modelo base.
        """
        super().__init__()
        # Cargar la configuración y el encoder del modelo base
        self.config = AutoConfig.from_pretrained(model_name)
        self.encoder = AutoModel.from_pretrained(model_name)
        
        # Capas personalizadas para la clasificación
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.config.hidden_size, 1)
        self.to(device)
        self.device = device


    def forward(self, input_ids, attention_mask=None, **kwargs):
        """
        Pase hacia adelante (forward pass). Acepta **kwargs para ignorar argumentos extra.
        """
        output = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled = self.dropout(output.last_hidden_state[:, 0])
        logits = self.classifier(pooled).squeeze(-1)
        return SequenceClassifierOutput(loss=None, logits=logits)

print("✅ Clase SoftBinaryClassifier definida correctamente.")

✅ Clase SoftBinaryClassifier definida correctamente.


In [25]:
# 1. Clase para clasificación multiclase (task 1.2)
class SoftMultiClassifier(nn.Module):
    MODEL_NAME = "xlm-roberta-large"
    def __init__(self, model_name: str = MODEL_NAME, num_labels: int = 4, dropout: float = 0.3):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
        self.encoder = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.config.hidden_size, num_labels)
        self.to(device)
        self.device = device

    def forward(self, input_ids, attention_mask=None, **kwargs):
        output = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled = self.dropout(output.last_hidden_state[:, 0])
        logits = self.classifier(pooled)
        return SequenceClassifierOutput(loss=None, logits=logits)

In [26]:
from typing import Dict, List # <-- AÑADE ESTA LÍNEA


class EXIST2025HybridSystem:
    """
    Sistema híbrido que combina Transformer local + LLM remoto para EXIST2025
    """
    def __init__(self, transformer_model_path: str = None):
        """
        Inicializador del sistema. Recibe la ruta al modelo afinado.
        """
        self.transformer_model = None
        self.transformer_tokenizer = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        retriever_1_1 = index_1_1.as_retriever(similarity_top_k=3)
        retriever_1_2 = index_1_2.as_retriever(similarity_top_k=3)
        retriever_1_3 = index_1_3.as_retriever(similarity_top_k=3)



        self.llm_builder = EXIST2025PromptBuilder(
        rag_index,
        language="es",
        retriever_1_1=retriever_1_1,
        retriever_1_2=retriever_1_2,
        retriever_1_3=retriever_1_3
    )

        # Configuración del ensemble
        self.weights = {
            'transformer': 0.6,  # Peso del transformer (más especializado)
            'llm': 0.4           # Peso del LLM (más generalista)
        }
        
        # Thresholds optimizados
        self.thresholds = {
            'confidence_min': 0.7,  # Confianza mínima para decisión unilateral
            'agreement_threshold': 0.8  # Umbral para considerar acuerdo entre modelos
        }
        
        if transformer_model_path:
            self.load_finetuned_transformer(transformer_model_path)

    def load_finetuned_transformer(self, model_path: str):
        """
        Carga el modelo transformer afinado de forma correcta.
        """
        print(f"🔧 Cargando modelo afinado desde: {model_path}")
        try:
            from safetensors.torch import load_file
            # Cargar tokenizer
            self.transformer_tokenizer = AutoTokenizer.from_pretrained(model_path)
            print("   ✅ Tokenizer cargado.")

            # Cargar config para saber qué modelo base se usó (ej: 'xlm-roberta-large')
            #config = AutoConfig.from_pretrained(model_path)
            #base_model_name = config._name_or_path
            #print(f"   ✅ Configuración leída. Modelo base: {base_model_name}")

            # Crear la "carcasa" del modelo usando el nombre del modelo base
            self.transformer_model = SoftBinaryClassifier()
            print("   ✅ Arquitectura SoftBinaryClassifier instanciada.")

            # Cargar los pesos entrenados en la carcasa
            weights_path = os.path.join(model_path, 'model.safetensors')
            state_dict = load_file(weights_path, device=str(self.device))
            self.transformer_model.load_state_dict(state_dict)
            print("   ✅ Pesos (safetensors) cargados en el modelo.")

            # Mover a GPU y poner en modo evaluación
            self.transformer_model.to(self.device)
            self.transformer_model.eval()
            print(f"   💻 Modelo listo en dispositivo: {self.transformer_model.device}")

        except Exception as e:
            print(f"   ❌ Error al cargar modelo: {e}")
            self.transformer_model = None
            self.transformer_tokenizer = None

    def predict_transformer(self, text: str) -> Dict:
        """
        Realiza una predicción usando el modelo Transformer cargado.
        """
        if self.transformer_model is None or self.transformer_tokenizer is None:
            return {'available': False, 'probability': 0.5}
        try:
            # Tokenizar y mover a GPU
            inputs = self.transformer_tokenizer(text, return_tensors="pt")
            inputs = {key: val.to(self.device) for key, val in inputs.items()}
            
            with torch.no_grad():
                outputs = self.transformer_model(**inputs)
                prob = torch.sigmoid(outputs.logits).item()
            
            prediction = "YES" if prob > 0.5 else "NO"
            confidence = abs(prob - 0.5) * 2
            
            return {
                'available': True, 'prediction': prediction,
                'probability': prob, 'confidence': confidence
            }
        except Exception as e:
            print(f"❌ Error en predicción Transformer: {e}")
            return {'available': False, 'probability': 0.5}
        

    def load_finetuned_transformer_task_1_2(self, model_path: str):
        print(f"🔧 Cargando modelo 1.2 desde: {model_path}")
        try:
            from safetensors.torch import load_file
            self.transformer_tokenizer_1_2 = AutoTokenizer.from_pretrained(model_path)
            self.transformer_model_1_2 = SoftMultiClassifier()
            weights_path = os.path.join(model_path, 'model.safetensors')
            state_dict = load_file(weights_path, device=str(self.device))
            self.transformer_model_1_2.load_state_dict(state_dict)
            self.transformer_model_1_2.to(self.device)
            self.transformer_model_1_2.eval()
            print("   ✅ Modelo 1.2 cargado y listo.")
        except Exception as e:
            print(f"   ❌ Error al cargar modelo 1.2: {e}")
            self.transformer_model_1_2 = None
            self.transformer_tokenizer_1_2 = None

    def predict_transformer_1_2(self, text: str) -> dict:
        if self.transformer_model_1_2 is None or self.transformer_tokenizer_1_2 is None:
            return {'available': False, 'prediction': None, 'confidence': 0.0}
        try:
            inputs = self.transformer_tokenizer_1_2(text, return_tensors="pt")
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            with torch.no_grad():
                outputs = self.transformer_model_1_2(**inputs)
                probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()[0]
            idx2label = {0: "DIRECT", 1: "REPORTED", 2: "JUDGEMENTAL",3: "NO"}
            pred_idx = int(np.argmax(probs))
            return {
                'available': True,
                'prediction': idx2label[pred_idx],
                'confidence': float(np.max(probs))
            }
        except Exception as e:
            print(f"❌ Error en predicción Transformer 1.2: {e}")
            return {'available': False, 'prediction': None, 'confidence': 0.0}

        
        
    def predict_llm(self, text: str, subtask: str = "1.1") -> Dict:
        """
        Predicción usando LLM remoto
        
        Args:
            text: Texto a analizar
            subtask: Subtarea EXIST2025 ("1.1", "1.2", "1.3")
            
        Returns:
            Dict con predicción y metadatos
        """
        try:
            # Construir prompt
            prompt = self.llm_builder.build_prompt(text, subtask)
            
            # Llamar a LLM
            response, model_used, tokens_in, tokens_out = prompt_decoupled(
                prompt, 
                model='mistral-nemo:latest'
            )
            
            if response and 'response' in response:
                raw_response = response['response'].strip()
                
                # Procesar respuesta según subtarea
                if subtask == "1.1":# Extraer JSON de la respuesta
                    try:
                        # Limpiar la respuesta para asegurar que sea un JSON válido
                        clean_response = raw_response.strip().replace("'", '"')
                        json_response = json.loads(clean_response)
                        
                        prediction = json_response.get("prediction", "NO").upper()
                        confidence = float(json_response.get("confidence", 0.5))
                        
                        return {
                            'available': True,
                            'prediction': prediction,
                            'confidence': confidence, # <-- AHORA LA CONFIANZA ES REAL
                            'raw_response': raw_response,
                            'model': model_used
                        }
                    except (json.JSONDecodeError, AttributeError):
                        # Si el LLM no devuelve un JSON, hacemos un fallback simple
                        prediction = "YES" if "YES" in raw_response.upper() else "NO"
                        return {'available': True, 'prediction': prediction, 'confidence': 0.75} # Fallback con confianza media
                
                elif subtask == "1.2":
                    # Extraer intención
                    for intention in ["DIRECT", "REPORTED", "JUDGEMENTAL"]:
                        if intention in raw_response.upper():
                            return {
                                'available': True,
                                'prediction': intention,
                                'confidence': 0.8,
                                'raw_response': raw_response,
                                'tokens_used': tokens_in + tokens_out,
                                'model': model_used
                            }
                
                elif subtask == "1.3":
                    # Extraer categorías
                    categories = []
                    for cat in ["IDEOLOGICAL-INEQUALITY", "STEREOTYPING-DOMINANCE", 
                               "OBJECTIFICATION", "SEXUAL-VIOLENCE", "MISOGYNY-NON-SEXUAL-VIOLENCE"]:
                        if cat in raw_response.upper():
                            categories.append(cat)
                    
                    return {
                        'available': True,
                        'prediction': categories,
                        'confidence': 0.8,
                        'raw_response': raw_response,
                        'tokens_used': tokens_in + tokens_out,
                        'model': model_used
                    }
            
            return {
                'available': False,
                'prediction': None,
                'confidence': 0.0,
                'error': 'No response from LLM'
            }
            
        except Exception as e:
            print(f"❌ Error en predicción LLM: {e}")
            return {
                'available': False,
                'prediction': None,
                'confidence': 0.0,
                'error': str(e)
            }
    
    def _ensemble_multiclass_decision(self, transformer_result: Dict, llm_result: Dict) -> Dict:
        """
        Lógica de ensemble para decisión multiclase (subtask 1.2).
        """
        t_avail = transformer_result.get('available', False)
        l_avail = llm_result.get('available', False)

        if t_avail and not l_avail:
            return transformer_result
        if l_avail and not t_avail:
            return llm_result
        if t_avail and l_avail:
            t_pred = transformer_result['prediction']
            t_conf = transformer_result['confidence']
            l_pred = llm_result['prediction']
            l_conf = llm_result['confidence']
            # Acuerdo
            if t_pred == l_pred:
                w_t = self.weights['transformer']
                w_l = self.weights['llm']
                avg_conf = (t_conf * w_t) + (l_conf * w_l)
                return {
                    'available': True,
                    'prediction': t_pred,
                    'confidence': min(avg_conf, 0.95),
                    'method': 'agreement',
                    'reasoning': f"Acuerdo en '{t_pred}'. Confianza ponderada: ({t_conf:.2f}*{w_t}) + ({l_conf:.2f}*{w_l}) = {avg_conf:.2f}"
                }
            # Desacuerdo
            else:
                if t_conf >= self.thresholds['confidence_min']:
                    return {**transformer_result, 'method': 'transformer_confident'}
                elif l_conf >= self.thresholds['confidence_min']:
                    return {**llm_result, 'method': 'llm_confident'}
                else:
                    return {**transformer_result, 'method': 'transformer_fallback'}
        # Ningún modelo disponible
        return {'available': False, 'prediction': None, 'confidence': 0.0, 'method': 'fallback'}

    def ensemble_prediction(self, text: str) -> dict:
        """
        Predicción ensemble combinando Transformer + LLM para 1.1 y 1.2.
        Devuelve SIEMPRE ambas predicciones (transformer y LLM) para cada subtask.
        """
        print(f"🔍 Analizando: {text[:50]}...")

        # Task 1.1 (binaria)
        transformer_1_1 = self.predict_transformer(text)
        llm_1_1 = self.predict_llm(text, "1.1")
        final_1_1 = self._ensemble_binary_decision(transformer_1_1, llm_1_1)

        # Task 1.2 (intención)
        transformer_1_2 = self.predict_transformer_1_2(text)
        llm_1_2 = self.predict_llm(text, "1.2")
        final_1_2 = self._ensemble_multiclass_decision(transformer_1_2, llm_1_2)

        # Task 1.3 (solo LLM por ahora)
        llm_1_3 = self.predict_llm(text, "1.3")

        # Compilar resultado completo, guardando TODO
        result = {
            'text': text,
            'subtask_1_1': {
                'ensemble': final_1_1,
                'transformer': transformer_1_1,
                'llm': llm_1_1
            },
            'subtask_1_2': {
                'ensemble': final_1_2,
                'transformer': transformer_1_2,
                'llm': llm_1_2
            },
            'subtask_1_3': {
                'llm': llm_1_3
            },
            'models_used': {
                'transformer_1_1': transformer_1_1.get('available', False),
                'llm_1_1': llm_1_1.get('available', False),
                'transformer_1_2': transformer_1_2.get('available', False),
                'llm_1_2': llm_1_2.get('available', False)
            },
            'ensemble_strategy_1_1': self._get_strategy_used(transformer_1_1, llm_1_1)
        }

        return result
    
    
    # Dentro de la clase EXIST2025HybridSystem:

    def _ensemble_binary_decision(self, transformer_result: Dict, llm_result: Dict) -> Dict:
        """
        Lógica de ensemble para decisión binaria con prints para depuración.
        """
        # --- Print Inicial ---
        print("\n🧠 --- Iniciando Lógica de Ensemble ---")
        print(f"   - Transformer: {transformer_result}")
        print(f"   - LLM: {llm_result}")
        
        # Extraer valores, manejando casos donde un modelo no esté disponible
        t_avail = transformer_result.get('available', False)
        l_avail = llm_result.get('available', False)

        # Caso 1: Solo Transformer disponible
        if t_avail and not l_avail:
            print("   -  decyzja: Solo Transformer disponible, usando su resultado.")
            return {
                'prediction': transformer_result['prediction'],
                'confidence': transformer_result['confidence'],
                'method': 'transformer_only',
                'reasoning': 'Solo Transformer disponible'
            }
        
        # Caso 2: Solo LLM disponible
        if l_avail and not t_avail:
            print("   - decyzja: Solo LLM disponible, usando su resultado.")
            return {
                'prediction': llm_result['prediction'],
                'confidence': llm_result['confidence'],
                'method': 'llm_only',
                'reasoning': 'Solo LLM disponible'
            }
        
        # Caso 3: Ambos modelos disponibles
        if t_avail and l_avail:
            t_pred = transformer_result['prediction']
            t_conf = transformer_result['confidence']
            l_pred = llm_result['prediction']
            l_conf = llm_result['confidence']
            
            # Acuerdo entre modelos
            if t_pred == l_pred:
                print(f"   - VEREDICTO: ✅ Acuerdo. Ambos modelos predicen '{t_pred}'.")
                w_t = self.weights['transformer']
                w_l = self.weights['llm']
                avg_confidence = (t_conf * w_t) + (l_conf * w_l)
                reasoning = f"Acuerdo en '{t_pred}'. Confianza ponderada: ({t_conf:.2f}*{w_t}) + ({l_conf:.2f}*{w_l}) = {avg_confidence:.2f}"
                print(f"     - {reasoning}")
                
                return { 'prediction': t_pred, 'confidence': min(avg_confidence, 0.95),
                        'method': 'agreement', 'reasoning': reasoning }
            
            # Desacuerdo
            else:
                print(f"   - VEREDICTO: ⚠️ Desacuerdo. Transformer -> '{t_pred}' ({t_conf:.2f}), LLM -> '{l_pred}' ({l_conf:.2f}).")
                
                if t_conf >= self.thresholds['confidence_min']:
                    reasoning = f'Transformer muy seguro ({t_conf:.2f} >= {self.thresholds["confidence_min"]}). Se prioriza su predicción.'
                    print(f"     - {reasoning}")
                    return { 'prediction': t_pred, 'confidence': t_conf * 0.9,
                            'method': 'transformer_confident', 'reasoning': reasoning }
                
                elif l_conf >= self.thresholds['confidence_min']:
                    reasoning = f'LLM muy seguro ({l_conf:.2f} >= {self.thresholds["confidence_min"]}). Se prioriza su predicción.'
                    print(f"     - {reasoning}")
                    return { 'prediction': l_pred, 'confidence': l_conf * 0.9,
                            'method': 'llm_confident', 'reasoning': reasoning }
                
                else:
                    reasoning = 'Ambos poco confiados. Prioridad a Transformer por ser especializado.'
                    print(f"     - {reasoning}")
                    return { 'prediction': t_pred, 'confidence': t_conf * 0.8,
                            'method': 'transformer_fallback', 'reasoning': reasoning }
        
        # Caso 4: Ningún modelo disponible
        print("   - decyzja: ❌ Ningún modelo disponible. Fallback conservador a 'NO'.")
        return {
            'prediction': 'NO', 'confidence': 0.0,
            'method': 'fallback', 'reasoning': 'Ningún modelo disponible'
        }
    
    def _get_strategy_used(self, transformer_result: Dict, llm_result: Dict) -> str:
        """Determinar qué estrategia se utilizó"""
        t_avail = transformer_result.get('available', False)
        l_avail = llm_result.get('available', False)
        
        if t_avail and l_avail:
            return 'hybrid_ensemble'
        elif t_avail:
            return 'transformer_only'
        elif l_avail:
            return 'llm_only'
        else:
            return 'fallback'
        
    def batch_predict(self, texts: List[str]) -> List[Dict]:
        """
        Predicción en lote
        
        Args:
            texts: Lista de textos a analizar
            
        Returns:
            Lista de resultados
        """
        results = []
        print(f"📊 Procesando {len(texts)} textos en modo híbrido...")
        
        for i, text in enumerate(texts):
            print(f"   {i+1}/{len(texts)}: {text[:30]}...")
            result = self.ensemble_prediction(text)
            results.append(result)
        
        return results
    
    def evaluate_performance(self, test_data: List[Dict]) -> Dict:
        """
        Evaluar rendimiento del sistema híbrido
        
        Args:
            test_data: Lista de diccionarios con 'text' y 'label'
            
        Returns:
            Métricas de evaluación
        """
        print("📊 Evaluando rendimiento del sistema híbrido...")
        
        predictions = []
        true_labels = []
        
        for item in test_data:
            result = self.ensemble_prediction(item['text'])
            pred = 1 if result['subtask_1_1']['prediction'] == 'YES' else 0
            predictions.append(pred)
            true_labels.append(item['label'])
        
        # Calcular métricas
        accuracy = accuracy_score(true_labels, predictions)
        f1 = f1_score(true_labels, predictions)
        precision = precision_score(true_labels, predictions)
        recall = recall_score(true_labels, predictions)
        
        metrics = {
            'accuracy': accuracy,
            'f1_score': f1,
            'precision': precision,
            'recall': recall,
            'total_samples': len(test_data)
        }
        
        print(f"✅ Evaluación completada:")
        print(f"   🎯 Accuracy: {accuracy:.4f}")
        print(f"   📈 F1-Score: {f1:.4f}")
        print(f"   📊 Precision: {precision:.4f}")
        print(f"   🔄 Recall: {recall:.4f}")
        
        return metrics

       

print("✅ Clase EXIST2025HybridSystem definida correctamente")
print("🚀 Listo para crear instancia del sistema híbrido")

✅ Clase EXIST2025HybridSystem definida correctamente
🚀 Listo para crear instancia del sistema híbrido


## 🧠 Justificación Profesional del Ensamble Ponderado (Transformer + LLM) en EXIST2025

El sistema híbrido implementado en este notebook combina modelos transformers locales y LLMs remotos mediante una estrategia de **ponderación** (ensemble weighted voting). Esta técnica se fundamenta en principios sólidos de la literatura de aprendizaje automático y procesamiento de lenguaje natural, y es ampliamente utilizada en sistemas de clasificación robustos y explicables.

---

### ¿Por qué es válida y útil la ponderación en ensambles?

1. **Complementariedad de Modelos**  
    - Los modelos **transformers finos** (como XLM-RoBERTa) están altamente especializados en el dominio y capturan patrones específicos del dataset, logrando alta precisión en ejemplos similares a los de entrenamiento.
    - Los **LLMs remotos** (como Mistral o Qwen) aportan razonamiento contextual, generalización y capacidad de adaptación a casos ambiguos o fuera de distribución, gracias a su entrenamiento masivo y prompts enriquecidos (RAG).

2. **Reducción de Sesgo y Varianza**  
    - El ensamble ponderado permite mitigar el sesgo de un modelo individual y reducir la varianza, combinando la fortaleza de cada enfoque.
    - Según [Dietterich, 2000](https://www.sciencedirect.com/science/article/pii/S0004370299001212), los métodos de ensamble mejoran la robustez y la generalización al aprovechar la diversidad de los modelos base.

3. **Confianza y Explicabilidad**  
    - La ponderación basada en la confianza de cada modelo permite priorizar la predicción más segura, aumentando la fiabilidad del sistema.
    - El razonamiento detrás de cada decisión queda documentado, facilitando la auditoría y la interpretación de resultados, aspecto clave en aplicaciones sensibles como la detección de sexismo.

4. **Adaptabilidad y Escalabilidad**  
    - El sistema puede ajustarse dinámicamente: si un modelo no está disponible o es poco confiable en un caso concreto, el otro puede tomar el liderazgo.
    - Esta flexibilidad es esencial en entornos de producción y en tareas multilingües o multi-dominio.

---

### Evidencia en la Literatura

- **Stacking y Weighted Voting**:  
  Técnicas de ensamble como el stacking y el weighted voting han demostrado mejoras sistemáticas en tareas de clasificación, especialmente cuando los modelos base son heterogéneos ([Opitz & Maclin, 1999](https://link.springer.com/article/10.1023/A:1009717407291)).
- **Aplicaciones en NLP**:  
  En tareas de NLP, la combinación de modelos finos y LLMs mediante ponderación ha sido validada en competiciones y benchmarks recientes, logrando mejores resultados que cualquier modelo individual ([Ruder et al., 2019](https://arxiv.org/abs/1906.08237)).

---

### Utilidad en el Contexto de EXIST2025

- **Precisión y Robustez**: El sistema híbrido maximiza la precisión en la detección y análisis de sexismo, adaptándose tanto a ejemplos claros como a casos ambiguos.
- **Explicabilidad**: Cada predicción viene acompañada de un razonamiento y una medida de confianza, alineándose con los principios de IA responsable.
- **Escalabilidad**: La arquitectura modular permite incorporar nuevos modelos o ajustar los pesos según métricas de validación.

---

### ¿Cómo se aplica la fórmula de ponderación en este sistema?

En este notebook, la decisión final del ensamble se calcula usando una **fórmula de combinación ponderada** de las confianzas de ambos modelos (transformer y LLM). Por ejemplo, para la tarea binaria (1.1), si ambos modelos están de acuerdo, la confianza final se calcula así:

- **Confianza final:**  
  `confianza_ensemble = (confianza_transformer * peso_transformer) + (confianza_llm * peso_llm)`

Donde los pesos (`peso_transformer` y `peso_llm`) reflejan la especialización y generalización de cada modelo (por defecto, 0.6 y 0.4 respectivamente).

- Si hay desacuerdo, el sistema prioriza el modelo con mayor confianza (si supera un umbral), o recurre al transformer como fallback si ambos tienen baja confianza.
- Para tareas multiclase (1.2), se aplica la misma lógica, combinando las confianzas y predicciones de ambos modelos.
- Para la tarea 1.3 (multietiqueta), actualmente solo se usa la predicción del LLM, pero la arquitectura permite extender la lógica de ensamble.

**Ejemplo de decisión (tarea 1.1):**
- Si ambos modelos predicen "YES" y sus confianzas son 0.9 (transformer) y 0.7 (LLM):  
  `confianza_ensemble = (0.9 * 0.6) + (0.7 * 0.4) = 0.54 + 0.28 = 0.82`

Esta confianza ponderada se reporta junto con la predicción y el razonamiento, permitiendo transparencia y trazabilidad en cada decisión.

---

**En resumen:**  
La técnica de ponderación en el ensamble híbrido no solo es válida, sino que representa el estado del arte en sistemas de clasificación robustos y explicables. Su implementación en este notebook garantiza un equilibrio óptimo entre especialización, generalización y transparencia, aportando valor científico y práctico a la solución del reto EXIST2025.

In [27]:
# 🚀 INICIALIZAR SISTEMA HÍBRIDO
# Instanciar pasando solo la ruta
hybrid_system = EXIST2025HybridSystem(transformer_model_path="../models/model_1_1_xml_roberta")
hybrid_system.load_finetuned_transformer_task_1_2("../models/model_1_2_xml_roberta")  # Cambia la ruta a tu modelo 1.2
# O instanciar pasando modelo y tokenizer ya cargados
# hybrid_system = EXIST2025HybridSystem(transformer_model=model, transformer_tokenizer=tokenizer)

🔧 Cargando modelo afinado desde: ../models/model_1_1_xml_roberta
   ✅ Tokenizer cargado.
   ✅ Arquitectura SoftBinaryClassifier instanciada.
   ✅ Pesos (safetensors) cargados en el modelo.
   💻 Modelo listo en dispositivo: cuda
🔧 Cargando modelo 1.2 desde: ../models/model_1_2_xml_roberta
   ✅ Modelo 1.2 cargado y listo.


In [28]:
# ...existing code...
text = " Me contaron que una amiga fue discriminada..."
resultado = hybrid_system.ensemble_prediction(text)
print(resultado)

🔍 Analizando:  Me contaron que una amiga fue discriminada......
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qdrant01.decoupled.ai/collections/task1_1_rag/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://qdrant01.decoupled.ai/collections/task1_1_rag/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qdrant01.decoupled.ai/collections/criterios_rag/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://qdrant01.decoupled.ai/collections/criterios_rag/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama01.decoupled.ai/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama01.decoupled.ai/api/chat "HTTP/1.1 200 O

In [ ]:
# ...existing code...
from pprint import pprint

print("\n🔎 SUBTAREA 1.1 (Binario):")
ensemble_1_1 = resultado['subtask_1_1']['ensemble']
print(f"  Predicción: {ensemble_1_1['prediction']} | Confianza: {ensemble_1_1['confidence']:.2f}")
print(f"  Método: {ensemble_1_1['method']}")
print(f"  Razonamiento: {ensemble_1_1.get('reasoning','')}")

print("\n🔎 SUBTAREA 1.2 (Intención):")

ensemble_1_2 = resultado['subtask_1_2']['ensemble']
print(f"  Predicción: {ensemble_1_2['prediction']} | Confianza: {ensemble_1_2['confidence']:.2f}")
print(f"  Método: {ensemble_1_2['method']}")
print(f"  Razonamiento: {ensemble_1_2.get('reasoning','')}")

print("\n🔎 SUBTAREA 1.3 (Categorías):")
llm_1_3 = resultado['subtask_1_3']['llm']
print(f"  Categorías detectadas: {llm_1_3['prediction']}")
print(f"  Confianza: {llm_1_3['confidence']:.2f}")
print(f"  Respuesta LLM: {llm_1_3.get('raw_response','')}")



# ...existing code...


🔎 SUBTAREA 1.1 (Binario):
  Predicción: NO | Confianza: 0.85
  Método: llm_confident
  Razonamiento: LLM muy seguro (0.95 >= 0.7). Se prioriza su predicción.

🔎 SUBTAREA 1.2 (Intención):
  Predicción: NO | Confianza: 0.82
  Método: transformer_confident
  Razonamiento: 

🔎 SUBTAREA 1.3 (Categorías):
  Categorías detectadas: []
  Confianza: 0.80
  Respuesta LLM: {"prediction": ["NO"], "confidence": 0.9}


## 📝 Nota sobre la Evaluación de las Predicciones

En esta celda, se generan las predicciones del modelo híbrido y del transformer utilizando el conjunto de desarrollo (`dev`). Estas predicciones se guardan en un archivo CSV .

### Evaluación de las Predicciones

La evaluación de las predicciones no se realiza en este notebook. En su lugar, se llevará a cabo en un notebook separado, utilizando la herramienta designada para la competición EXIST2025. Esto asegura que el proceso de evaluación esté desacoplado de la generación de predicciones, permitiendo mayor flexibilidad y organización en el flujo de trabajo.



In [30]:
import json
import pandas as pd

# 1. Cargar el archivo de desarrollo
with open("../data/goldDEV/EXIST2025_dev.json", "r", encoding="utf-8") as f:
    dev_data = json.load(f)
print(f"📚 Datos de desarrollo cargados: {len(dev_data)} ejemplos")

# 2. Extraer los campos requeridos
dev_samples = []
for k, v in dev_data.items():
    if 'tweet' in v:
        dev_samples.append({
            'id_EXIST': v['id_EXIST'],
            'lang': v['lang'],
            'tweet': v['tweet']
        })
df_pred = pd.DataFrame(dev_samples)
print(df_pred.head())

# 3. Realizar predicciones y guardar las originales
binary_predictions = []
intention_predictions = []
category_predictions = []

for _, row in df_pred.iterrows():
    tweet = row["tweet"]
    result = hybrid_system.ensemble_prediction(tweet)
    binary_pred = result['subtask_1_1']['ensemble']['prediction']
    intention_pred = result['subtask_1_2']['ensemble']['prediction']
    category_pred = result['subtask_1_3']['llm']['prediction']
    binary_predictions.append(binary_pred)
    intention_predictions.append(intention_pred)
    category_predictions.append(category_pred)



📚 Datos de desarrollo cargados: 1038 ejemplos
  id_EXIST lang                                              tweet
0   300001   es  @Fichinescu La comunidad gamer es un antro de ...
1   300002   es  @anacaotica88 @MordorLivin No me acuerdo de lo...
2   300003   es  @cosmicJunkBot lo digo cada pocos dias y lo re...
3   300004   es  Also mientras les decia eso la señalaba y deci...
4   300005   es  And all people killed,  attacked, harassed by ...
🔍 Analizando: @Fichinescu La comunidad gamer es un antro de misó...
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qdrant01.decoupled.ai/collections/task1_1_rag/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://qdrant01.decoupled.ai/collections/task1_1_rag/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Reque

In [31]:
df_pred["binary_prediction_raw"] = binary_predictions
df_pred["intention_prediction_raw"] = intention_predictions
df_pred["category_prediction_raw"] = category_predictions

df_pred.to_csv("predicciones_exist2025_dev_raw.csv", index=False, encoding="utf-8")
print("✅ Archivo predicciones_exist2025_dev_raw.csv exportado (predicciones originales).")

# 4. Aplicar la lógica de consistencia para el dataset final
binary_final = []
intention_final = []
category_final = []

for b, i, c in zip(binary_predictions, intention_predictions, category_predictions):
    if b == "NO":
        binary_final.append("NO")
        intention_final.append("NO")
        category_final.append(["NO"])
    else:
        binary_final.append(b)
        intention_final.append(i)
        category_final.append(c)


df_pred["binary_prediction"] = binary_final
df_pred["intention_prediction"] = intention_final
df_pred["category_prediction"] = category_final

# Guardar dataset final en CSV y JSON
df_pred[["id_EXIST", "lang", "tweet", "binary_prediction", "intention_prediction", "category_prediction"]].to_csv(
    "predicciones_exist2025_dev_final.csv", index=False, encoding="utf-8"
)
print("✅ Archivo predicciones_exist2025_dev_final.csv exportado (dataset final).")


✅ Archivo predicciones_exist2025_dev_raw.csv exportado (predicciones originales).
✅ Archivo predicciones_exist2025_dev_final.csv exportado (dataset final).
